# Big Data Platform
## Assignment 2: MapReduce

**By:**  

John Doe, 300123123  
Jane Doe, 200123123

<br><br>

**The goal of this assignment is to:**
- Understand and practice the details of MapReduceEngine

**Instructions:**
- Students will form teams of two people each, and submit a single homework for each team.
- The same score for the homework will be given to each member of your team.
- Your solution is in the form of a Jupyter notebook file (with extension ipynb).
- Images/Graphs/Tables should be submitted inside the notebook.
- The notebook should be runnable and properly documented. 
- Please answer all the questions and include all your code.
- You are expected to submit a clear and pythonic code.
- You can change functions signatures/definitions.

**Submission:**
- Submission of the homework will be done via Moodle by uploading a Jupyter notebook.
- The homework needs to be entirely in English.
- The deadline for submission is on Moodle.
- Late submission won't be allowed.
  
  
- In case of identical code submissions - both groups will get a Zero. 
- Some groups might be selected randomly to present their code.

**Requirements:**  
- Python 3.6 should be used.  
- You should implement the algorithms by yourself using only basic Python libraries (such as numpy,pandas,etc.)

<br><br><br><br>

**Grading:**
- Q1 - 5 points - Initial Steps
- Q2 - 50 points - MapReduceEngine
- Q3 - 30 points - Implement the MapReduce Inverted index of the JSON documents
- Q4 - 5 points - Testing Your MapReduce
- Q5 - 10 points - Final Thoughts 

`Total: 100`

**Prerequisites**

In [18]:
# example
import itertools
import sqlite3
import traceback
#!pip install --quiet zipfile36

**Imports**

In [19]:
# general
import os
import shutil
import random
import warnings
import concurrent
from pathlib import Path
# ml
import numpy as np
import pandas as pd
import logging

# visual
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display

**Hide Warnings**

In [20]:
warnings.filterwarnings('ignore')

**Disable Autoscrolling**

In [21]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

**Set Random Seed**

In [22]:
random.seed(123)

<br><br><br><br>
# Question 1
# Initial Steps

Write Python code to create 20 different CSV files in this format:  `myCSV[Number].csv`, where each file contains 10 records. 

The schema is `(‘firstname’,’secondname’,city’)`  

Values should be randomly chosen from the lists: 
- `firstname` : `[John, Dana, Scott, Marc, Steven, Michael, Albert, Johanna]`  
- `city` : `[New York, Haifa, München, London, Palo Alto,  Tel Aviv, Kiel, Hamburg]`  
- `secondname`: any value  

In [35]:
# my_path = "/Users/eyalmichaeli/Desktop/School/Master's"
my_path = "/Users/mymac"
exercise_path = "IDC_masters/big_data_platforms_ex2"

path = Path(my_path) / Path(exercise_path)

In [36]:
firstname = ['John', 'Dana', 'Scott', 'Marc', 'Steven', 'Michael', 'Albert', 'Johanna']
city = ['NewYork', 'Haifa', 'Munchen', 'London', 'PaloAlto', 'TelAviv', 'Kiel', 'Hamburg']
secondname = ['Lennon', 'McCartney', 'Starr', 'Harrison', 'Ono', 'Sutcliffe', 'Epstein', 'Preston']

csvs_path = Path(path / "csvs")
csvs_path.mkdir(parents=True, exist_ok=True)
for i in range(1, 21):
    temp_df = pd.DataFrame({"firstname": np.random.choice(firstname, 10),
                            "secondname": np.random.choice(secondname, 10),
                            "city": np.random.choice(city, 10),
                            })
    temp_df.to_csv(str(csvs_path / f"myCSV{i}.csv"))

print("Created 20 CSV files")

Created 20 CSV files


Use python to Create `mapreducetemp` and `mapreducefinal` folders

In [37]:
mapreducetemp_folder = Path(path / "mapreducetemp")
mapreducetemp_folder.mkdir(parents=True, exist_ok=True)

mapreducefinal_folder = Path(path / "mapreducefinal")
mapreducefinal_folder.mkdir(parents=True, exist_ok=True)

print("Created folders")

Created folders


<br><br><br>
# Question 2
## MapReduceEngine

Write Python code to create an SQLite database with the following table

`TableName: temp_results`   
`schema: (key:TEXT,value:TEXT)`

In [38]:

conn = None
try:
    conn = sqlite3.connect(str(path / "temp_results.db"))
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS temp_results (key, value);")

except Exception as e:
    traceback.print_exc()

# finally:
#     cursor.close()
#     if conn:
#         conn.close()

1. **Create a Python class** `MapReduceEngine` with method `def execute(input_data, map_function, reduce_function)`, such that:
    - `input_data`: is an array of elements
    - `map_function`: is a pointer to the Python function that returns a list where each entry of the form (key,value) 
    - `reduce_function`: is pointer to the Python function that returns a list where each entry of the form (key,value)

<br><br>

**Implement** the following functionality in the `execute(...)` function:

<br>

1. For each key  from the  input_data, start a new Python thread that executes map_function(key) 
<br><br>
2. Each thread will store results of the map_function into mapreducetemp/part-tmp-X.csv where X is a unique number per each thread. 
<br><br>
3. Keep the list of all threads and check whether they are completed.
<br><br>
4. Once all threads completed, load content of all CSV files into the temp_results table in SQLite.

    Remark: Easiest way to loop over all CSV files and load them into Pandas first, then load into SQLite
    `data = pd.read_csv(path to csv)`
    `data.to_sql(‘temp_results’,sql_conn, if_exists=’append’,index=False)`
<br><br>

5. **Write SQL statement** that generates a sorted list by key of the form `(key, value)` where value is concatenation of ALL values in the value column that match specific key. For example, if table has records
<table>
    <tbody>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV1.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">Dana</td>
                <td style="text-align:center">myCSV5.csv</td>
            </tr>
            <tr>
                <td style="text-align:center">John</td>
                <td style="text-align:center">myCSV7.csv</td>
            </tr>
    </tbody>
</table>

    Then SQL statement will return `(‘John’,’myCSV1.csv, myCSV7.csv’)`  
    Remark: use GROUP_CONCAT and also GROUP BY ORDER BY
<br><br><br>
6. **Start a new thread** for each value from the generated list in the previous step, to execute `reduce_function(key,value)` 
<br>    
7. Each thread will store results of reduce_function into `mapreducefinal/part-X-final.csv` file  
<br>
8. Keep list of all threads and check whether they are completed  
<br>
9. Once all threads completed, print on the screen `MapReduce Completed` otherwise print `MapReduce Failed` 



In [39]:
# implement all of the class here


class MapReduceEngine:

    def __init__(self, conn):
        self.conn = conn

    def execute(self, input_data, map_function, reduce_function):
        thread_list = list()
        csvs_paths = list()
        for csv_key in input_data:
            exec = concurrent.futures.ThreadPoolExecutor()
            t = exec.submit(map_function, csv_key)
            threads_returns = t.result()
            csv_index = input_data.index(csv_key)  # an index of the relative csv in the input_array
            csv_path = f'{mapreducetemp_folder}/part-tmp-{csv_index}.csv'
            csvs_paths.append(csv_path)
            pd.DataFrame(threads_returns).to_csv(csv_path,
                                                 header=['key', 'value'],
                                                 index=False)
            thread_list.append(t)

        # wait until the threads are completed
        exec.shutdown(wait=True)

        # Once all threads completed, load content of all CSV files into the temp_results table in Sqlite
        for path_to_csv in csvs_paths:
            data = pd.read_csv(path_to_csv)
            data.to_sql(name='temp_results', con=self.conn, if_exists='append', index=False)


        results_df = pd.read_sql_query("SELECT key,GROUP_CONCAT(value) as value FROM temp_results GROUP BY key ORDER BY key", conn)
        for res_i in range(len(results_df)):
            key = results_df["key"].iloc[res_i]
            value = results_df["value"].iloc[res_i]
            exec = concurrent.futures.ThreadPoolExecutor()
            t = exec.submit(reduce_function, key, value)
            t_results = t.result() # t_results is one list, in which the 1st index is the key and the 2nd is a concat of all of the files it appears in.
            csv_path = f'{mapreducefinal_folder}/part-{res_i}-final.csv'
            csvs_paths.append(csv_path)
            pd.DataFrame({'key': t_results[0], 'value': t_results[1]}, index=[0]).to_csv(csv_path,
                                                 index=False)
            thread_list.append(t)



        # wait until the threads are completed
        if exec.shutdown(wait=True):
            cursor.close()
            if conn:
                conn.close()
            return 'MapReduce Completed'
        cursor.close()
        if conn:
            conn.close()
        return 'MapReduce Failed'




<br><br><br><br>

# Question 3
## Implement the MapReduce Inverted index of the JSON documents

Implement a function `inverted_map(document_name)` which reads the CSV document from the local disc and return a list that contains entries of the form (key_value, document name).

For example, if myCSV4.csv document has values like:  
`{‘firstname’:’John’,‘secondname’:’Rambo’,‘city’:’Palo Alto’}`

Then `inverted_map(‘myCSV4.csv’)` function will return a list:  
`[(‘firstname_John’,’ myCSV4.csv’),(‘secondname_Rambo’,’ myCSV4.csv’), (‘city_Palo Alto’,’ myCSV4.csv’)]`

In [40]:
def inverted_map(document_name):
    df = pd.read_csv(path / 'csvs'/ document_name)
    lst = list()
    for col in ['firstname', 'secondname', 'city']:
        lst.extend([(str(col) + '_' + str(df[col].iloc[i]), document_name) for i in range(len(df))])
    return lst

Write a reduce function `inverted_reduce(value, documents)`, where the field “documents” contains a list of all CSV documents per given value.
This list might have duplicates.
Reduce function will return new list without duplicates.

For example,
calling the function `inverted_reduce(‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’)`
will return a list `[‘firstname_Albert’,’myCSV2.csv, myCSV5.csv,myCSV2.csv’]`

In [41]:
def inverted_reduce(value, documents):
    documents_formatted = (", ").join(set(documents.split(',')))
    return [value, documents_formatted]

<br><br><br><br>
# Question 4
## Testing Your MapReduce

**Create Python list** `input_data` : `[‘myCSV1.csv’,.. ,‘myCSV20.csv’]`

In [42]:
input_data = [f'myCSV{i}.csv' for i in range(1,21)]

**Submit MapReduce as follows:**

In [44]:
mapreduce = MapReduceEngine(conn=conn)
status = mapreduce.execute(input_data, inverted_map, inverted_reduce)
print(status)

ProgrammingError: Cannot operate on a closed database.

Make sure that `MapReduce Completed` should be printed and `mapreducefinal` folder should contain the result files.

**Use python to delete all temporary data from mapreducetemp folder and delete SQLite database:**

In [34]:
# delete all temp data from mapreducetemp
shutil.rmtree(path / 'mapreducetemp')

# delete the SQLite database
try:
    conn = sqlite3.connect(str(path / "temp_results.db"))
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS temp_results;")

except Exception as e:
    traceback.print_exc()

finally:
    cursor.close()
    if conn:
        conn.close()

<br><br><br><br>

# Question 5
# Final Thoughts

The phase where `MapReduceEngine` reads all temporary files generated by maps and sort them to provide each reducer a specific key is called the **shuffle step**.

Please explain **clearly** what would be the main problem of MapReduce when processing Big Data, if there is no shuffle step at all, meaning reducers will directly read responses from the mappers.

Basically, the shuffling phase is where the data is being grouped by the key and summed accordingly. The input the shufflers get is the broken down version of the data (the output of the mappers), and the output is the data aggregated. In terms of volume, the shufflers significantly reduce the amount of data that is being moved forward. In the word counting example, shufflers reduce the data from the amount of words in the origin text to the amount of distinct words.
<br> If it wasn't for the shuffling phase, the reducers would have to deal with a lot more data as input

<br><br><br><br>
Good Luck :)